# All Functions Used For Optimization

## K-Means

In [11]:
def weighted_kmeans(full_data, k, weight, seed = 3):
    data = [list(full_data['LON']), list(full_data['LAT']), list(full_data[weight])]
    random.seed(seed)
    n = len(data[0]) # number of census tracts
    stop = False
    labels = np.zeros(n) # assigns each centroid to a cluster
    centroids = np.zeros([k, 2]) # centroid for each cluster
    count = 0

    # randomly initialize centroids to be somewhere in the STL City region (between min and max of data)
    '''
    for i in range(k):
        centroids[i][0] = round(random.uniform(np.min(data[0]), np.max(data[0])), 6)
        centroids[i][1] = round(random.uniform(np.min(data[1]), np.max(data[1])), 6)
    '''
    # randomly initialize centroids to be existing census tracts
    for i in range(k):
        new_tract = random.randint(0,n-1)
        centroids[i][0] = data[0][new_tract]
        centroids[i][1] = data[1][new_tract]
        
    # iterate until labels do not change (or at least twice)
    while stop == False or count < 2:
        count = count + 1
        old_labels = labels

        # Calculating labels by finding nearest centroid for each census tract
        for i in range(n): # for each tract
            closest_distance = float('inf')
            tract_center = np.array([data[0][i], data[1][i]])
            for j in range(k): # for each centroid/cluster
                centroid = np.array([centroids[j][0], centroids[j][1]])
                dist = np.linalg.norm(tract_center - centroid)
                if (dist < closest_distance):
                    closest_distance = dist
                    labels[i] = j

        # Check if any of the labels changed
        all_same = True
        for i in range(n):
            if (labels[i] != old_labels[i]):
                all_same = False
        if all_same == True:
            stop = True

        # updating centroid locations as the (weighted?) mean of each census tract in its cluster
        new_centroids = np.zeros([k,2])
        for i in range(k): # for each cluster
            weights = []
            cluster = []
            for j in range(n): # for each tract
                if (labels[j] == i):
                    weights.append(data[2][j]) # weights will not all sum to 1 (different number in each cluster)
                    cluster.append(np.array([data[0][j], data[1][j]]))

            new_x = [point[0] for point in cluster]
            new_y = [point[1] for point in cluster]
            normalized_weights = np.array(weights)/np.sum(weights)

            if len(new_x) > 1: # if there are points in the cluster
                new_centroids[i][0] = np.dot(normalized_weights, new_x)
                new_centroids[i][1] = np.dot(normalized_weights, new_y)
            else:
                new_centroids[i][0] = centroids[i][0]
                new_centroids[i][1] = centroids[i][1]

        centroids = new_centroids
        
        return centroids, labels

## Linear Programming 

In [2]:
def distances_to_nearest_stop(tract_locations, metro_locations):
    
    shortest_distance = np.zeros(len(tract_locations))
    metro_locations = [[metro_locations[i], metro_locations[i + 1]] for i in range(0, len(metro_locations), 2)] # reshape back into 2d array
    
    for i in range(len(tract_locations)):
        cent = np.array([tract_locations['LON'][i],tract_locations['LAT'][i]])
        closest_distance = float('inf') # initialize to infinity
        for j in range(len(metro_locations)):
            metro = np.array(metro_locations[j])
            distance = np.linalg.norm(cent-metro)
            if (distance < closest_distance):
                closest_distance = distance
        shortest_distance[i] = closest_distance
    
    return shortest_distance

In [3]:
def orth_dist(x):
    x = [[x[i], x[i + 1]] for i in range(0, len(x), 2)] # reshape back into 2d array
    longs = np.array([s[0] for s in x]).reshape(-1,1)
    lats = np.array([s[1] for s in x])
    model = LinearRegression().fit(longs,lats)
    b = model.intercept_
    m = model.coef_[0]
    dists = 0
    for stop in x:
        n = abs(-1*m*stop[0] + stop[1] - b)
        d = np.sqrt(m**2+1)
        dists = dists + n/d
    return dists

In [4]:
def distances_between_stops(metro_locations):
    metro_locations = [[metro_locations[i], metro_locations[i + 1]] for i in range(0, len(metro_locations), 2)] # reshape back into 2d array
    shortest_distance = np.zeros(len(metro_locations))
    total_distance = 0
    for i in range(len(metro_locations)):
        metro_current = np.array([metro_locations[i][0],metro_locations[i][1]])
        closest_distance = float('inf') # initialize to infinity
        for j in range(len(metro_locations)):
            if (i != j):
                metro_compared = np.array(metro_locations[j])
                distance = np.linalg.norm(metro_current-metro_compared)
                if (distance < closest_distance):
                    closest_distance = distance
        total_distance = total_distance + closest_distance
    return total_distance

In [5]:
def distance_to_mean_x(metro_locations):
    metro_locations = [[metro_locations[i], metro_locations[i + 1]] for i in range(0, len(metro_locations), 2)] # reshape back into 2d array
    longs = np.array([s[0] for s in metro_locations])
    x_avg = np.mean(longs)
    deviation_from_avg = 0
    for i in range(len(metro_locations)):
        deviation_from_avg = deviation_from_avg + np.abs(x_avg - metro_locations[i][0])
    return deviation_from_avg

In [6]:
def fun(new_stops):
    #weights = full_data['race weight']
    weights = np.ones(len(full_data))
    dist_to_stops = np.dot(weights, distances_to_nearest_stop(full_data, new_stops))
    linearity = orth_dist(new_stops) 
    dist_btw_stops = distances_between_stops(new_stops)
    distance_to_mean = distance_to_mean_x(new_stops)
    return 5*dist_to_stops + 0*linearity - 1.0*dist_btw_stops + 2*distance_to_mean

In [7]:
def linear_programming(full_data):
    k = 12
    n = len(full_data)
    x0 = []
    for i in range(k):
        new_tract = random.randint(0,n-1)
        x0.append(full_data['LON'][new_tract])
        x0.append(full_data['LAT'][new_tract])
    
    lat_bounds = (min(full_data['LAT']),max(full_data['LAT']))
    lon_bounds = (min(full_data['LON']),max(full_data['LON']))
    bnds = [val for pair in zip([lon_bounds]*12, [lat_bounds]*12) for val in pair]
    
    result = minimize(fun, x0, bounds=bnds)
    all_centroids = result.x
    lp_results = [[all_centroids[i], all_centroids[i + 1]] for i in range(0, len(all_centroids), 2)]
    return lp_results

In [22]:
def graph_from_lp(tract_locations, metro_locations):
    
    shortest_distance = np.zeros(len(tract_locations))
    cluster_ids = np.zeros(len(tract_locations))
    
    for i in range(len(tract_locations)):
        cent = np.array([tract_locations['LON'][i],tract_locations['LAT'][i]])
        closest_distance = float('inf') # initialize to infinity
        closest_stop = float('inf')
        for j in range(len(metro_locations)):
            metro = np.array(metro_locations[j])
            distance = np.linalg.norm(cent-metro)
            if (distance < closest_distance):
                closest_distance = distance
                closest_stop = j
        shortest_distance[i] = closest_distance
        cluster_ids[i] = closest_stop
      
    return cluster_ids

## Modularity Maximization

In [20]:
def mod_max_weighted(graph, k):
    tolerance = 1e-5
    orig = graph.copy()
    to_return = graph.copy()
    q = 0
    q_new = 0
    partition = [orig]
    new_partition = []

    while (len(partition) < k):
        for G in partition:
            # compute modularity matrix B
            G_nodes = list(G.nodes())
            s = np.zeros(len(G_nodes))
            A = nx.to_numpy_array(G, weight='weights')
            degs = list(dict(G.degree()).values())
            B = np.zeros(np.shape(A))
            n = len(G_nodes)
            m = len(G.edges())
            for i in range(n):
                for j in range(n):
                    B[i][j] = A[i][j] - degs[i]*degs[j]/(2*m)
            # find leading eigenvector u of B
            val, vec = np.linalg.eig(B)
            idx = np.argsort(val)[-1]
            largest_vec = vec[:,idx]
            
            # divide nodes into two groups by sign of leading eigenvector
            for i in range(len(largest_vec)):
                if largest_vec[i].real < np.mean(largest_vec):
                    s[i] = -1
                else:
                    s[i] = 1
            
            # create new subgraphs
            n1 = [G_nodes[i] for i in range(n) if s[i] == 1]
            n2 = [G_nodes[i] for i in range(n) if s[i] == -1]
            if len(n1) >= 1:
                new_partition.append(G.subgraph(n1))
            if len(n2) >= 1:
                new_partition.append(G.subgraph(n2))
            
            # update q_new
            q_new = q_new + B.sum()
        
        
        # either stop or re-partition graph
        q_new = q_new / (2*m)
        if abs(q_new - q) > tolerance and q_new < q:
            return partition
        
        else:
            q = q_new
            q_new = 0 
            partition = new_partition
            new_partition = []

    if len(partition) > k: # (this happens if k is odd)
        num_to_combine = len(partition) - k # we need to re-combine this many graphs to get back to k subgraphs
        pairs = [(partition[i], partition[i + 1]) for i in range(0, len(partition), 2)]
        to_test = list(itertools.combinations(pairs, num_to_combine))
        
        # test modularity of all possible combinations to make partition have length k
        best_modularity = 0
        test_idx = 0
        best_test_idx = 0
        for test in to_test:
            partition_copy = partition.copy()
            # remove the pair from the partition and add back in the original before it split into that pair
            for pair in test:
                partition_copy.remove(pair[0])
                partition_copy.remove(pair[1])
                nodes_to_find = list(set(pair[0].nodes()).union(pair[1].nodes()))
                back_together = orig.subgraph(nodes_to_find)
                partition_copy.append(back_together)
            # re-compute modularity and compare to best modularity
            communities = []
            for clust in partition_copy:
                nodes_in_clust = {n for n in clust.nodes()}
                communities.append(nodes_in_clust)
            modularity = nx.community.modularity(orig, communities)
            if modularity > best_modularity:
                best_modularity = modularity
                best_test_idx = test_idx     
            test_idx = test_idx + 1
        
        # choose the test with the best modularity and combine the pairs in that test for the final result
        final_partition = partition.copy()
        for pair in to_test[best_test_idx]:
            final_partition.remove(pair[0])
            final_partition.remove(pair[1])
            nodes_to_find = list(set(pair[0].nodes()).union(pair[1].nodes()))
            back_together = orig.subgraph(nodes_to_find)
            final_partition.append(back_together)
            
    else: # if k was even
        final_partition = partition

    value_dict = dict()
    idx = 0
    for sub in final_partition:
        for n in sub.nodes():
            value_dict[n] = idx
        idx = idx + 1
    nx.set_node_attributes(to_return, value_dict, 'cluster')
    return to_return

# Graph Results

In [12]:
def calculate_centroid(group):
    return MultiPoint(group.geometry.values).centroid

In [27]:
def graph_results(res, title, graph_filename = "capstone.gexf", centroids_filename = "data\Centroids.shp"):
    
    graph = nx.read_gexf("capstone.gexf")
    colors = [graph.nodes[node]['cluster'] for node in res.nodes()]
    labels = [graph.nodes[node]['label'] for node in res.nodes()]
    gdf = gpd.GeoDataFrame(graph.nodes(data=True), columns=['geometry', 'attributes'])
    # Add colors and labels to the GeoDataFrame
    gdf['color'] = colors
    gdf['NAME'] = labels

    CentroidsInfo = gpd.read_file(centroids_filename)
    # Add color information to centroids GeoDataFrame
    centroids_gdf = gdf.merge(CentroidsInfo, on='NAME')
    centroids_gdf = gpd.GeoDataFrame(centroids_gdf.rename(columns={'geometry_y': 'geometry'}))
    centroids_gdf = centroids_gdf.drop(columns=['geometry_x','attributes'])
    centroids_gdf = centroids_gdf.to_crs('EPSG:4326')


    cent = centroids_gdf.groupby('color').apply(calculate_centroid)
    newPlan = gpd.GeoDataFrame(geometry = cent)

    centroids_gdf['geometry'] = centroids_gdf['geometry'].apply(lambda x: Point(x))
    new_centroids_gdf = gpd.GeoDataFrame(centroids_gdf, geometry='geometry', crs='EPSG:4326')

    # Save to file
    newPlan.to_file(r"data\newPlanClust.shp", crs="EPSG:4326")
    centroids_gdf.to_file(r"data\centroidsColoredByCommunity.shp",crs="EPSG:4326")
    
    # ------------------------------------- SAVING FILES ABOVE, GRAPHING BELOW --------------------------------------------
    
    
    # Read the tracts shapefile
    tracts = gpd.read_file(r"data\census tracts\CensusTractsStl.shp")

    # Plotting the graph and tracts polygons
    fig, ax = plt.subplots(figsize=(10, 10))

    # Plot tracts polygons
    tracts.plot(ax=ax, color='lightgray', edgecolor='black', alpha=0.5)

    coloredCentroids = gpd.read_file(r"data\centroidsColoredByCommunity.shp")
    color_dict = {0:'lightgreen', 1:'lightblue', 2:'lavender', 3:'orange', 4:'red', 5:'purple', 6:'yellow', 7:'green', 8:'blue', 9:'magenta', 10:'cyan', 11:'gray'}
    cmap = ListedColormap([color_dict[i] for i in range(len(color_dict))])

    coloredCentroids.head()


    # Plot centroids from centroidsColoredByCommunity.shp with colormap
    for _, row in coloredCentroids.iterrows():
        centroid = row['geometry'].centroid  # Get centroid of the polygon
        color = color_dict[row['color']]
        ax.plot(centroid.x, centroid.y, marker='o', color=color, markersize=5)  # Use colormap for centroids

    mm_centroids = list()
    # Plot centroids of polygons from newPlanClust.shp
    newPlan = gpd.read_file(r"data\newPlanClust.shp")
    for _, row in newPlan.iterrows():
        centroid = row['geometry'].centroid  # Get centroid of the polygon
        mm_centroids.append([centroid.x, centroid.y])
        ax.plot(centroid.x, centroid.y, marker='*', color='black', markersize=15)

    ax.set_title(f'Results of {title}')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    plt.savefig(f'{title}.png')
    plt.tight_layout()
    plt.show()
    
    
    to_return = np.array(mm_centroids)
    
    return to_return

In [21]:
def graph_results_lp(res, lp_stops, title, graph_filename = "capstone.gexf", centroids_filename = "data\Centroids.shp"):
    
    graph = nx.read_gexf("capstone.gexf")
    colors = [graph.nodes[node]['cluster'] for node in res.nodes()]
    labels = [graph.nodes[node]['label'] for node in res.nodes()]
    gdf = gpd.GeoDataFrame(graph.nodes(data=True), columns=['geometry', 'attributes'])
    # Add colors and labels to the GeoDataFrame
    gdf['color'] = colors
    gdf['NAME'] = labels

    CentroidsInfo = gpd.read_file(centroids_filename)
    # Add color information to centroids GeoDataFrame
    centroids_gdf = gdf.merge(CentroidsInfo, on='NAME')
    centroids_gdf = gpd.GeoDataFrame(centroids_gdf.rename(columns={'geometry_y': 'geometry'}))
    centroids_gdf = centroids_gdf.drop(columns=['geometry_x','attributes'])
    centroids_gdf = centroids_gdf.to_crs('EPSG:4326')


    cent = centroids_gdf.groupby('color').apply(calculate_centroid)
    newPlan = gpd.GeoDataFrame(geometry = cent)

    centroids_gdf['geometry'] = centroids_gdf['geometry'].apply(lambda x: Point(x))
    new_centroids_gdf = gpd.GeoDataFrame(centroids_gdf, geometry='geometry', crs='EPSG:4326')

    # Save to file
    newPlan.to_file(r"data\newPlanClust.shp", crs="EPSG:4326")
    centroids_gdf.to_file(r"data\centroidsColoredByCommunity.shp",crs="EPSG:4326")
    
    # ------------------------------------- SAVING FILES ABOVE, GRAPHING BELOW --------------------------------------------
    
    
    # Read the tracts shapefile
    tracts = gpd.read_file(r"data\census tracts\CensusTractsStl.shp")

    # Plotting the graph and tracts polygons
    fig, ax = plt.subplots(figsize=(10, 10))

    # Plot tracts polygons
    tracts.plot(ax=ax, color='lightgray', edgecolor='black', alpha=0.5)

    coloredCentroids = gpd.read_file(r"data\centroidsColoredByCommunity.shp")
    color_dict = {0:'lightgreen', 1:'lightblue', 2:'lavender', 3:'orange', 4:'red', 5:'purple', 6:'yellow', 7:'green', 8:'blue', 9:'magenta', 10:'cyan', 11:'gray'}
    cmap = ListedColormap([color_dict[i] for i in range(len(color_dict))])

    coloredCentroids.head()


    # Plot centroids from centroidsColoredByCommunity.shp with colormap
    for _, row in coloredCentroids.iterrows():
        centroid = row['geometry'].centroid  # Get centroid of the polygon
        color = color_dict[row['color']]
        ax.plot(centroid.x, centroid.y, marker='o', color=color, markersize=5)  # Use colormap for centroids

    # Plot original lp stops
    longs = [i[0] for i in lp_stops]
    lats = [i[1] for i in lp_stops]
    
    for stop in range(len(lp_stops)):
        ax.plot(longs[stop], lats[stop], marker='*', color='black', markersize=15)


    ax.set_title(f'Results of {title}')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    plt.savefig(f'{title}.png')
    plt.tight_layout()
    plt.show()

# Evaluation

In [25]:
def dist_to_nearest_stop_eval(centroid_locations, metro_locations):
    shortest_distance = np.zeros(len(centroid_locations))
    for i in range(len(centroid_locations)):
        cent = np.array([centroid_locations['LON'][i],centroid_locations['LAT'][i]])
        closest_distance = float('inf') # initialize to infinity
        for j in range(len(metro_locations)):
            metro = np.array([metro_locations[j][0],metro_locations[j][1]])
            diff_latlon = cent-metro
            diff_miles = np.copy(diff_latlon)
            diff_miles[0] = 69*diff_latlon[0]
            diff_miles[1] = 54.6*diff_latlon[1]
            distance = np.linalg.norm(diff_miles)
            if (distance < closest_distance):
                closest_distance = distance
        shortest_distance[i] = closest_distance
    
    return shortest_distance
